In [78]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [79]:
movie_data = open('movie_data_new.csv','wb')
skipped = open('skipped_urls_imdb_data.txt','wb')
df = pd.read_csv('book_names.csv', names=['Book Name'])
book_names = df['Book Name'].values
skipped_urls = []

In [80]:
def getnum(s):
    return ''.join(s.split(','))

def get_content(book_name, ID, link1, link2, f, a_href):
    try:
        movie_page = requests.get(link1, timeout=30)
        soup = BeautifulSoup(movie_page.content, "lxml")

        budget = ''
        gross = ''
        opening = ''

        li = soup.find_all('div',id='tn15content')
        if(len(li)!=0):
            for row in li:
                #print row
                re_obj = re.search("Budget<\/h5>[\s]*\$(.*?)\s", str(row))
                if(re_obj!=None):
                    budget = getnum(re_obj.group(1))
                re_obj = re.search("Gross<\/h5>[\s]*\$(.*?)\s", str(row))
                if(re_obj!=None):
                    gross = getnum(re_obj.group(1))
                re_obj = re.search("Opening Weekend<\/h5>[\s]*\$(.*?)\s", str(row))
                if(re_obj!=None):
                    opening = getnum(re_obj.group(1))
            
        movie_page = requests.get(link2, timeout=30)
        soup = BeautifulSoup(movie_page.content, "lxml")
        
        releasedate = ''
        
        li = soup.find_all('a',href=a_href)
        if(len(li)!=0):
            for row in li:
                #print row
                re_obj = re.search(">(.*)(USA)", str(row))
                if(re_obj!=None):
                    releasedate = getnum(re_obj.group(1))
                    releasedate = releasedate[:-1].strip()

        #print(book_name+'\t'+ID+'\t'+link1+'\t'+budget+'\t'+gross+'\t'+opening+'\t'+releasedate)
        print(budget+'\t'+gross+'\t'+opening+'\t'+releasedate)
        f.write(book_name+'\t'+ID+'\t'+link1+'\t'+budget+'\t'+gross+'\t'+opening+'\t'+releasedate+'\n')
            
    except Exception as exception:
        skipped_urls.append(ID)
        print(exception)

In [81]:
iter=0

for b in book_names:
    try:
        query = "http://www.omdbapi.com/?t="+b+"&y=&plot=full&r=json"
        res = requests.get(query)
        text = res.json()
        url1 = "http://www.imdb.com/title/" + text['imdbID'] + "/"
        url2 = "http://www.imdb.com/title/" + text['imdbID'] + "/business"
        a_href = "/title/" + text['imdbID'] + "/releaseinfo?ref_=tt_ov_inf"
        iter += 1
        print iter
#         print url1
#         print url2
#         print a_href
        get_content(b, text['imdbID'], url2, url1, movie_data, a_href)
    except Exception as exception:
        skipped.write(b+'\n')
        
    
movie_data.close()

for i in skipped_urls:
    skipped.write(i+'\n')

skipped.close()

1
150000000	292004738	77108414	11 July 2007
2
150000000	292004738	77108414	11 July 2007
3
37000000	191465414	69637740	21 November 2008
4
150000000	290013036	102335066	18 November 2005
5
130000000	249358727	93687367	4 June 2004
6
125000000	317557891	90294621	16 November 2001
7
100000000	261970615	88357488	15 November 2002
8
250000000	301959197	77835727	15 July 2009
9
100000000	75021527	23239907	15 December 2006
10
85000000	57010853	682504	23 December 2005
11
109000000	100446895	38329160	21 November 2003
12
125000000	381011219	169189427	15 July 2011
13
125000000	381011219	169189427	15 July 2011
14
180000000	291709845	65556312	9 December 2005
15
180000000	70083519	25783232	7 December 2007
16
95000000	88761720		12 February 2010
17
35000000	22913677	6169030	9 April 2004
18
			28 June 1940
19
			
20
28000000	10907291	5324240	5 February 2016
21
2000000	13129846		16 March 1963
22
	82506325	11457353	15 December 2006
23
20000000	67406573	16300155	18 April 2003
24
78000000	408010692	152535747	23 

KeyboardInterrupt: 